<br> 
<center><img src="https://i.imgur.com/hkb7Bq7.png" width="500"></center>


### **Prof. José Manuel Magallanes, PhD**

* Professor, Departamento de Ciencias Sociales, Pontificia Universidad Católica del Perú, [jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe)

* Visiting Professor, Evans School of Public Policy and Governance / Senior Data Science Fellow, eScience Institute, University of Washington, [magajm@uw.edu](mailto:magajm@uw.edu)
_____

_____


# Introduction to Python

### Using Python for Pre Processing

In the session we will see the use of Python to:

1. Collect data tables into Python

2. Preprocess both tables:
    * Subset data
    * Fix column names
    * Look for non-standar missing values
    * Clean cell values
    * Format data types


3. Merge both tables


4. Prepare a file for further analysis



## 1. Collect data tables into Python

* **From a file**:

In [1]:
# Location of data file
linkFile="https://github.com/eScienceWinterSchool/data/raw/master/HDI_2018.xlsx"

Reading in a table from a file using pandas, since it is an Excel file, I requires **openpyxl**:

In [ ]:
# available
!pip show openpyxl

If not available, please go to Anaconda and install it. Once installed, or if available, continue:

In [2]:
# choose the right function:
import pandas as pd

hdiFile=pd.read_excel(linkFile)

## Pre Processing

### Subset data

Just keep what you need; let's check the data head and tail:

In [3]:
hdiFile.head(10)

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,SDG3,NaN,SDG4.3,NaN,SDG4.6,NaN,SDG8.5,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
4,HDI rank,Country,(index value),NaN,(years),NaN,(years),NaN,(years),NaN,(2011 PPP $),NaN,NaN,NaN,NaN
5,NaN,NaN,2018,NaN,2018,NaN,2018,a,2018,a,2018,NaN,2018,NaN,2017
6,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,Norway,0.953688,NaN,82.271,NaN,18.06082,b,12.566818,NaN,68058.61613,NaN,5,NaN,1
8,2,Switzerland,0.945936,NaN,83.63,NaN,16.20882,NaN,13.380812,NaN,59374.73403,NaN,8,NaN,2
9,3,Ireland,0.942473,NaN,82.103,NaN,18.79326,b,12.526295,c,55659.67902,NaN,9,NaN,3


In [4]:
hdiFile.tail(65)

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
204,..,Somalia,..,NaN,57.068,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
205,..,Tuvalu,..,NaN,..,NaN,12.30895,NaN,..,NaN,5408.949247,NaN,..,NaN,..
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,Human development groups,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,NaN,Very high human development,0.891786,NaN,79.505042,NaN,16.361804,NaN,12.043619,NaN,40111.566426,NaN,—,NaN,—
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,Column 2: UNDESA (2019b).,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,NaN,Column 3: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,Column 4: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,"Column 5: World Bank (2019a), IMF (2019) and U...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# subsetting
hdiRaw=hdiFile.iloc[7:206,:].copy()
hdiRaw

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
7,1,Norway,0.953688,NaN,82.271,NaN,18.06082,b,12.566818,NaN,68058.61613,NaN,5,NaN,1
8,2,Switzerland,0.945936,NaN,83.63,NaN,16.20882,NaN,13.380812,NaN,59374.73403,NaN,8,NaN,2
9,3,Ireland,0.942473,NaN,82.103,NaN,18.79326,b,12.526295,c,55659.67902,NaN,9,NaN,3
10,4,Germany,0.938785,NaN,81.18,NaN,17.09638,NaN,14.13215,NaN,46945.9499,NaN,15,NaN,4
11,4,"Hong Kong, China (SAR)",0.938809,NaN,84.687,NaN,16.51223,NaN,12.03813,NaN,60220.79676,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,..,Monaco,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
202,..,Nauru,..,NaN,..,NaN,11.25936,e,..,NaN,17312.58793,NaN,..,NaN,..
203,..,San Marino,..,NaN,..,NaN,15.1112,NaN,..,NaN,..,NaN,..,NaN,..
204,..,Somalia,..,NaN,57.068,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..


### Fix column names

Notice the column names are in different places.

In [6]:
# save column names 
RealHeaders1=hdiFile.iloc[4,:2].to_list()
RealHeaders2=hdiFile.iloc[3,2:].to_list()

In [7]:
# all column names
RealHeaders=RealHeaders1+RealHeaders2
RealHeaders

['HDI rank',
 'Country',
 'Human development index (HDI) ',
 nan,
 'Life expectancy at birth',
 nan,
 'Expected years of schooling',
 nan,
 'Mean years of schooling',
 nan,
 'Gross national income (GNI) per capita',
 nan,
 'GNI per capita rank minus HDI rank',
 nan,
 'HDI rank']

Notice the repeated column names. Let's avoid that:

In [8]:
hdiRaw=hdiRaw.iloc[:,1:]
hdiRaw

,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
7,Norway,0.953688,NaN,82.271,NaN,18.06082,b,12.566818,NaN,68058.61613,NaN,5,NaN,1
8,Switzerland,0.945936,NaN,83.63,NaN,16.20882,NaN,13.380812,NaN,59374.73403,NaN,8,NaN,2
9,Ireland,0.942473,NaN,82.103,NaN,18.79326,b,12.526295,c,55659.67902,NaN,9,NaN,3
10,Germany,0.938785,NaN,81.18,NaN,17.09638,NaN,14.13215,NaN,46945.9499,NaN,15,NaN,4
11,"Hong Kong, China (SAR)",0.938809,NaN,84.687,NaN,16.51223,NaN,12.03813,NaN,60220.79676,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Monaco,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
202,Nauru,..,NaN,..,NaN,11.25936,e,..,NaN,17312.58793,NaN,..,NaN,..
203,San Marino,..,NaN,..,NaN,15.1112,NaN,..,NaN,..,NaN,..,NaN,..
204,Somalia,..,NaN,57.068,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..


Then, let's just use the column names needed:

In [9]:
hdiRaw.columns=RealHeaders[1:]
hdiRaw

,Country,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
7,Norway,0.953688,NaN,82.271,NaN,18.06082,b,12.566818,NaN,68058.61613,NaN,5,NaN,1
8,Switzerland,0.945936,NaN,83.63,NaN,16.20882,NaN,13.380812,NaN,59374.73403,NaN,8,NaN,2
9,Ireland,0.942473,NaN,82.103,NaN,18.79326,b,12.526295,c,55659.67902,NaN,9,NaN,3
10,Germany,0.938785,NaN,81.18,NaN,17.09638,NaN,14.13215,NaN,46945.9499,NaN,15,NaN,4
11,"Hong Kong, China (SAR)",0.938809,NaN,84.687,NaN,16.51223,NaN,12.03813,NaN,60220.79676,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Monaco,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
202,Nauru,..,NaN,..,NaN,11.25936,e,..,NaN,17312.58793,NaN,..,NaN,..
203,San Marino,..,NaN,..,NaN,15.1112,NaN,..,NaN,..,NaN,..,NaN,..
204,Somalia,..,NaN,57.068,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..


We still have column names with missing values, let's get rid of those:

In [11]:
BetterHeaders=hdiRaw.columns.dropna().to_list()
BetterHeaders

['Country',
 'Human development index (HDI) ',
 'Life expectancy at birth',
 'Expected years of schooling',
 'Mean years of schooling',
 'Gross national income (GNI) per capita',
 'GNI per capita rank minus HDI rank',
 'HDI rank']

In [12]:
#subsetting again
hdiRaw=hdiRaw.loc[:,BetterHeaders]
hdiRaw.head()

,Country,Human development index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
7,Norway,0.953688,82.271,18.06082,12.566818,68058.61613,5,1
8,Switzerland,0.945936,83.63,16.20882,13.380812,59374.73403,8,2
9,Ireland,0.942473,82.103,18.79326,12.526295,55659.67902,9,3
10,Germany,0.938785,81.18,17.09638,14.13215,46945.9499,15,4
11,"Hong Kong, China (SAR)",0.938809,84.687,16.51223,12.03813,60220.79676,5,6


Notice above that the columns:
* Have acronyms in parenthesis.
* Have spaces between words.
* Have many words.
* A couple of the represent rankings.

Let's see what can be done:

In [13]:
# bye anything between parenthesis

hdiRaw.columns=hdiRaw.columns.str.replace('\(.+\)',"", regex=True)
#then
hdiRaw.columns

Index(['Country', 'Human development index  ', 'Life expectancy at birth',
       'Expected years of schooling', 'Mean years of schooling',
       'Gross national income  per capita',
       'GNI per capita rank minus HDI rank', 'HDI rank'],
      dtype='object')

In [14]:
# capitalize each word with title()
hdiRaw.columns=hdiRaw.columns.str.title()
#then
hdiRaw.columns

Index(['Country', 'Human Development Index  ', 'Life Expectancy At Birth',
       'Expected Years Of Schooling', 'Mean Years Of Schooling',
       'Gross National Income  Per Capita',
       'Gni Per Capita Rank Minus Hdi Rank', 'Hdi Rank'],
      dtype='object')

Now, it is time to decide how the we want as the shorter column name:

* Same title without spaces:

In [15]:
hdiRaw.columns.str.replace(" ",'',regex=False)

Index(['Country', 'HumanDevelopmentIndex', 'LifeExpectancyAtBirth',
       'ExpectedYearsOfSchooling', 'MeanYearsOfSchooling',
       'GrossNationalIncomePerCapita', 'GniPerCapitaRankMinusHdiRank',
       'HdiRank'],
      dtype='object')

* New acronyms (step by step):

In [16]:
# each column names splitted:
[name.split() for name in hdiRaw.columns]

[['Country'],
 ['Human', 'Development', 'Index'],
 ['Life', 'Expectancy', 'At', 'Birth'],
 ['Expected', 'Years', 'Of', 'Schooling'],
 ['Mean', 'Years', 'Of', 'Schooling'],
 ['Gross', 'National', 'Income', 'Per', 'Capita'],
 ['Gni', 'Per', 'Capita', 'Rank', 'Minus', 'Hdi', 'Rank'],
 ['Hdi', 'Rank']]

In [17]:
# first letter of each word
[[word[0] for word in name.split()] for name in hdiRaw.columns]

[['C'],
 ['H', 'D', 'I'],
 ['L', 'E', 'A', 'B'],
 ['E', 'Y', 'O', 'S'],
 ['M', 'Y', 'O', 'S'],
 ['G', 'N', 'I', 'P', 'C'],
 ['G', 'P', 'C', 'R', 'M', 'H', 'R'],
 ['H', 'R']]

In [18]:
# final result
[''.join([word[0] for word in name.split()]) for name in hdiRaw.columns]

['C', 'HDI', 'LEAB', 'EYOS', 'MYOS', 'GNIPC', 'GPCRMHR', 'HR']

Let's keep the first alternative:

In [19]:
hdiRaw.columns=hdiRaw.columns.str.replace(" ",'',regex=False)

Finally, this will get rid of the column names with the word _Rank_:

In [20]:
import re

[header for header in hdiRaw.columns if re.search('Rank|rank',header)]

['GniPerCapitaRankMinusHdiRank', 'HdiRank']

In [21]:
byHeaders=[header for header in hdiRaw.columns if re.search('Rank|rank',header)]
# then
hdiRaw.drop(columns=byHeaders, inplace=True)
#finally
hdiRaw


,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita
7,Norway,0.953688,82.271,18.06082,12.566818,68058.61613
8,Switzerland,0.945936,83.63,16.20882,13.380812,59374.73403
9,Ireland,0.942473,82.103,18.79326,12.526295,55659.67902
10,Germany,0.938785,81.18,17.09638,14.13215,46945.9499
11,"Hong Kong, China (SAR)",0.938809,84.687,16.51223,12.03813,60220.79676
...,...,...,...,...,...,...
201,Monaco,..,..,..,..,..
202,Nauru,..,..,11.25936,..,17312.58793
203,San Marino,..,..,15.1112,..,..
204,Somalia,..,57.068,..,..,..


### Look for non-standar missing values

First check a cell that is full of non-word/non-digit characters:

In [22]:
# full match!
[hdiRaw.iloc[:,1].str.fullmatch("\W+",na=False)]

[7      False
 8      False
 9      False
 10     False
 11     False
        ...  
 201     True
 202     True
 203     True
 204     True
 205     True
 Name: HumanDevelopmentIndex, Length: 199, dtype: bool]

The above result is telling you if whether there is or there is not a full match (True/False). You can use that to keep the rows where this is _True_:

In [23]:
hdiRaw.iloc[:,1][hdiRaw.iloc[:,1].str.fullmatch("\W+",na=False)]

200    ..
201    ..
202    ..
203    ..
204    ..
205    ..
Name: HumanDevelopmentIndex, dtype: object

Now you a cell full of messy values. Let's see what we find in the next column:

In [24]:
hdiRaw.iloc[:,2][hdiRaw.iloc[:,2].str.fullmatch("\W+",na=False)]

201    ..
202    ..
203    ..
205    ..
Name: LifeExpectancyAtBirth, dtype: object

This works, but it is not a good strategy if we are going to do the same for several columns. In this case, it would be good to **customize** a function and use a loop. 

Let's prepare the function first:

In [26]:
# function name = lambda INPUT:F(INPUT)

GetBadCells=lambda Column:Column[Column.str.fullmatch("\W+",na=False)]

In [27]:
# let's see:

GetBadCells(hdiRaw.iloc[:,2])

201    ..
202    ..
203    ..
205    ..
Name: LifeExpectancyAtBirth, dtype: object

It is important to verify what structure the function returns:

In [28]:
type(GetBadCells(hdiRaw.iloc[:,2]))

pandas.core.series.Series

Now, you can output a different structure:

In [29]:
# as a set, so you get rid of duplicates:
GetBadCells=lambda x:set(x[x.str.fullmatch("\W+",na=False)])

In [30]:
# then
GetBadCells(hdiRaw.iloc[:,2])

{'..'}

In [31]:
# what about turning the set into a list:
GetBadCells=lambda x:list(set(x[x.str.fullmatch("\W+",na=False)]))

In [32]:
GetBadCells(hdiRaw.iloc[:,2])

['..']

Now we will prepare a  **loop**. A loop is needed when you need to apply some code, usually a function, in a repetitive way. 

We need to apply the function we created in every column but the first one. How many columns we have:

In [33]:
hdiRaw.shape[1]

6

Then, we can use **range** to create a list of column positions:

In [34]:
range(1,hdiRaw.shape[1])

range(1, 6)

In [35]:
# if you do not use list() you will not see the result
# but it exists.
list(range(1,hdiRaw.shape[1]))

[1, 2, 3, 4, 5]

Let's see how a loop actually works:

In [36]:
for value in range(1,hdiRaw.shape[1]):
    print(value)

1
2
3
4
5


Now let's combine the function and the loop:

In [37]:
badValues=[] # an empty list
for COL in range(1,hdiRaw.shape[1]):
    bads=GetBadCells(hdiRaw.iloc[:,COL])
    badValues.extend(bads)
    badValues=list(set(badValues))

The object _badValues_ stores the cell values that are not numbers (like '2.3'). Let's see what we got: 

In [38]:
badValues

['..']

This means that the people who created this data set used ".." to represent **missing values**. 

All we have done has the purpose of replacing that string by the actual missing values representation (**None**):

In [39]:
hdiRaw.replace(to_replace=badValues,
                value=None,inplace=True)
hdiRaw

,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita
7,Norway,0.953688,82.271,18.06082,12.566818,68058.61613
8,Switzerland,0.945936,83.63,16.20882,13.380812,59374.73403
9,Ireland,0.942473,82.103,18.79326,12.526295,55659.67902
10,Germany,0.938785,81.18,17.09638,14.13215,46945.9499
11,"Hong Kong, China (SAR)",0.938809,84.687,16.51223,12.03813,60220.79676
...,...,...,...,...,...,...
201,Monaco,None,None,None,None,None
202,Nauru,None,None,11.25936,None,17312.58793
203,San Marino,None,None,15.1112,None,None
204,Somalia,None,57.068,None,None,None


### Cleaning cell values

Do the current cell values have issues?

In [40]:
# get rid of leading and trailing spaces in text cells
hdiRaw.Country=hdiRaw.Country.str.strip()

do the numbers are useful for computing?

In [41]:
hdiRaw.iloc[:,1:].max()

HumanDevelopmentIndex              0.953688
LifeExpectancyAtBirth                84.687
ExpectedYearsOfSchooling           22.10372
MeanYearsOfSchooling               14.13215
GrossNationalIncomePerCapita    110488.7358
dtype: object

Notice the function **describe** should produce statistical results:

In [42]:
hdiRaw.describe(include='all')

,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita
count,199,189.000000,191.000,193.00000,189.000000,191.00000
unique,199,189.000000,191.000,193.00000,189.000000,191.00000
top,Norway,0.953688,82.271,18.06082,12.566818,68058.61613
freq,1,1.000000,1.000,1.00000,1.000000,1.00000


But, you are not getting those because they lack format.

In [44]:
hdiClean=hdiRaw.copy()

### Formatting

Check the data types:

In [46]:
hdiClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 7 to 205
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Country                       199 non-null    object
 1   HumanDevelopmentIndex         189 non-null    object
 2   LifeExpectancyAtBirth         191 non-null    object
 3   ExpectedYearsOfSchooling      193 non-null    object
 4   MeanYearsOfSchooling          189 non-null    object
 5   GrossNationalIncomePerCapita  191 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB


Country can remain as an object (text), but not the rest.

* **Formatting into numeric type**:

In [47]:
# as easy as:
hdiClean[hdiClean.columns[1:]]=hdiClean.iloc[:,1:].apply(pd.to_numeric)

In [48]:
#recheck
hdiClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 7 to 205
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country                       199 non-null    object 
 1   HumanDevelopmentIndex         189 non-null    float64
 2   LifeExpectancyAtBirth         191 non-null    float64
 3   ExpectedYearsOfSchooling      193 non-null    float64
 4   MeanYearsOfSchooling          189 non-null    float64
 5   GrossNationalIncomePerCapita  191 non-null    float64
dtypes: float64(5), object(1)
memory usage: 9.5+ KB


In [49]:
# recheck
hdiClean.describe(include='all')

,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita
count,199,189.000000,191.000000,193.000000,189.000000,191.000000
unique,199,NaN,NaN,NaN,NaN,NaN
top,Norway,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.713449,72.414157,13.211942,8.613036,18368.072291
std,NaN,0.150802,7.509645,2.935362,3.082290,19627.352750
min,NaN,0.376591,52.805000,5.000380,1.585574,659.732263
25%,NaN,0.595670,67.227500,11.304630,6.348000,4009.432932
50%,NaN,0.727787,73.752000,13.067556,9.017541,11610.908180
75%,NaN,0.830096,77.701500,15.230000,11.288111,26535.420700


In [50]:
hdiFormat=hdiClean.copy()

## Case 3: Integrating and Saving

In [51]:
demoFormat=pd.read_pickle("demoFormat.pkl")

This should be an easy step:

In [52]:
hdiFormat.merge(demoFormat)

,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties
0,Norway,0.953688,82.271,18.060820,12.566818,68058.616130,Full democracy,9.75,10.00,9.64,10.00,10.00,9.12
1,Switzerland,0.945936,83.630,16.208820,13.380812,59374.734030,Full democracy,8.90,9.58,8.93,7.78,9.38,8.82
2,Ireland,0.942473,82.103,18.793260,12.526295,55659.679020,Full democracy,9.00,10.00,7.86,8.33,9.38,9.41
3,Germany,0.938785,81.180,17.096380,14.132150,46945.949900,Full democracy,8.67,9.58,8.21,8.33,8.13,9.12
4,Australia,0.938379,83.281,22.103720,12.683050,44097.019340,Full democracy,8.90,10.00,8.57,7.78,8.75,9.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Mali,0.427206,58.893,7.603360,2.352954,1965.385142,Authoritarian,3.48,2.42,0.00,5.56,5.63,3.82
142,Burundi,0.422882,61.247,11.304630,3.124365,659.732263,Authoritarian,2.13,0.00,0.00,3.89,5.00,1.76
143,Chad,0.401176,53.977,7.465364,2.409497,1715.568235,Authoritarian,1.67,0.00,0.00,2.22,3.75,2.35
144,Central African Republic,0.380662,52.805,7.568360,4.282000,776.675996,Authoritarian,1.43,1.25,0.00,1.67,1.88,2.35


Notice you have lost countries:

In [53]:
len(hdiFormat),len(demoFormat)

(199, 167)

In [54]:
hdidem=hdiFormat.merge(demoFormat)
# result:
hdidem

,Country,HumanDevelopmentIndex,LifeExpectancyAtBirth,ExpectedYearsOfSchooling,MeanYearsOfSchooling,GrossNationalIncomePerCapita,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties
0,Norway,0.953688,82.271,18.060820,12.566818,68058.616130,Full democracy,9.75,10.00,9.64,10.00,10.00,9.12
1,Switzerland,0.945936,83.630,16.208820,13.380812,59374.734030,Full democracy,8.90,9.58,8.93,7.78,9.38,8.82
2,Ireland,0.942473,82.103,18.793260,12.526295,55659.679020,Full democracy,9.00,10.00,7.86,8.33,9.38,9.41
3,Germany,0.938785,81.180,17.096380,14.132150,46945.949900,Full democracy,8.67,9.58,8.21,8.33,8.13,9.12
4,Australia,0.938379,83.281,22.103720,12.683050,44097.019340,Full democracy,8.90,10.00,8.57,7.78,8.75,9.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Mali,0.427206,58.893,7.603360,2.352954,1965.385142,Authoritarian,3.48,2.42,0.00,5.56,5.63,3.82
142,Burundi,0.422882,61.247,11.304630,3.124365,659.732263,Authoritarian,2.13,0.00,0.00,3.89,5.00,1.76
143,Chad,0.401176,53.977,7.465364,2.409497,1715.568235,Authoritarian,1.67,0.00,0.00,2.22,3.75,2.35
144,Central African Republic,0.380662,52.805,7.568360,4.282000,776.675996,Authoritarian,1.43,1.25,0.00,1.67,1.88,2.35


____
____


### <font color="red">Saving File to Disk</font>

#### For future use in Python:

In [55]:
hdidem.to_pickle("hdidemocia.pkl")
# you will need: DF=pd.read_pickle("hdidemocia.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://...../hdidemocia.pkl"),compression=None)

#### For future  use in R:

In [58]:
!pip show rpy2

Name: rpy2
Version: 3.5.7
Summary: Python interface to the R language (embedded R)
Home-page: 
Author: 
Author-email: Laurent Gautier <lgautier@gmail.com>
License: GPLv2+
Location: /Users/JoseManuel/opt/anaconda3/envs/WinterSchool-3_11/lib/python3.11/site-packages
Requires: cffi, jinja2, pytz, tzlocal
Required-by: 


In [59]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(hdidem,file="hdidem.RDS")

#In R, you call it with: DF = readRDS("hdidem.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://...../hdidem.RDS")

<rpy2.rinterface_lib.sexp.NULLType object at 0x1624ad710> [0]